In [9]:
#IMPORTS

import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

import requests
from bs4 import BeautifulSoup
import time

In [10]:
df = pd.DataFrame(columns=['href_list']) # creates master dataframe 

# Chromedriver Path
PATH = "/Users/javm/Desktop/chromedriver_mac_arm64/chromedriver"

# this creates/opens Chrome browser
driver = webdriver.Chrome(PATH)

#creating list of href
href_list = []

for listings in range(1,21):
    page_num = str(listings)
    url = 'https://www.barnesandnoble.com/b/books/_/N-1z13w9sZ1sZ29Z8q8?Nrpp=20&page=' + page_num
    driver.get(url)
    # freeze/halt python so url loads completely
    driver.implicitly_wait(60)
    
    #find hrefs
    div_elements = driver.find_elements(By.CSS_SELECTOR, "div.product-shelf-title.product-info-title.pt-xs")
    for div_element in div_elements:
        elements = div_element.find_elements(By.TAG_NAME, "a")
        for element in elements:
            href = element.get_attribute('href')
            href_list.append(href)

    urls_df = pd.DataFrame(href_list, columns=['href_list']) # creates master dataframe
    df = pd.concat([df, urls_df], ignore_index=True)
    
driver.close()


/var/folders/6g/zn0bw_6s1ws2qwbb44k4ppkw0000gn/T/ipykernel_19295/243403321.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [11]:
len(href_list)

400

In [12]:
# Chromedriver Path
PATH = "/Users/javm/Desktop/chromedriver_mac_arm64/chromedriver"

# this creates/opens Chrome browser
driver = webdriver.Chrome(PATH)

book_details = []

for index, href in enumerate(href_list):
    driver.get(href)
    driver.implicitly_wait(60)

    # find titles
    titles = driver.find_elements(By.TAG_NAME, 'h1')
    title_list = []
    for t in range(len(titles)):
        title_list.append(titles[t].text)

    if not title_list:
        continue

    # find author
    author_list = []
    try:
        author_elems = driver.find_elements(By.CSS_SELECTOR, '.contributors.pdp-book-author a')
        for author_elem in author_elems:
            author_list.append(author_elem.text)
    except NoSuchElementException:
        author_list = [np.nan]

    # find publisher
    try:
        publisher_span = driver.find_element(By.CSS_SELECTOR, 'span[itemprop="publisher"]')
        publisher = publisher_span.text
    except NoSuchElementException:
        publisher = np.nan

    # find blurb
    try:
        parent_div = driver.find_element(By.XPATH, '//*[@id="overviewSection"]')
        blurb_text = parent_div.text
    except NoSuchElementException:
        blurb_text = np.nan
    
    book = [title_list[1] if len(title_list) > 1 else title_list[0], author_list, publisher, blurb_text]
    book_details.append(book)

driver.quit()

BOOK_DF_1 = pd.DataFrame(book_details, columns=['title', 'authors', 'publisher', 'blurb'])



/var/folders/6g/zn0bw_6s1ws2qwbb44k4ppkw0000gn/T/ipykernel_19295/1683694768.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [13]:
BOOK_DF_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      400 non-null    object
 1   authors    400 non-null    object
 2   publisher  400 non-null    object
 3   blurb      397 non-null    object
dtypes: object(4)
memory usage: 12.6+ KB


In [14]:
BOOK_DF_1.to_csv('books_1.csv', index = False)

In [15]:
BOOK_DF_1

,title,authors,publisher,blurb
0,Generación idiota: Una crítica al adolescentrismo,"[, Agustin Laje]",HarperCollins Mexico,Overview\nNotes From Your Bookseller\nEl contr...
1,Spare: En la sombra,"[, Prince Harry, The Duke of Sussex]",PRH Grupo Editorial,Overview\nNotes From Your Bookseller\nLa contr...
2,Volver a empezar / It Starts with Us (Spanish ...,"[, Colleen Hoover]",Planeta Publishing Corporation,Overview\nNotes From Your Bookseller\n¡Colleen...
3,¡Vámonos a la Estufa! con Janet Jauja Cocina M...,"[, Kushner, Janet]",Larousse,"Overview\nNotes From Your Bookseller\nJauja, l..."
4,RUGE: O espera a ser devorado (Spanish Edition),"[, Daniel Habif]",Editorial Planeta Mexicana S.A. de C.V.,Overview\nNotes From Your Bookseller\nCon su c...
...,...,...,...,...
395,Autocuración,"[, Jeff Rediger]","Urano Publishing, Inc.",Overview\nUn psiquiatra de Harvard desvela las...
396,Indira (Indira Ramos 3),"[, Santiago Díaz]",Penguin Random House Grupo Editorial España,Overview\nIndira Ramos en la mayor encrucijada...
397,Nuestro secreto,"[, Alessandra Neymar]",Amazon Digital Services LLC - Kdp,Overview\nUna historia de BAJO EL CIELO PÚRPUR...
398,Mucho después de medianoche,"[, , Ray Bradbury, Simon Saito (Translator)]",Planeta Publishing Corporation,Overview\nVeintidós cuentos para leer mucho de...
